In [1]:
import pandas as pd
import os
import string
from tqdm import tqdm
from itertools import permutations, combinations
from zipfile import ZipFile
import re

from nltk.corpus import stopwords
# nltk.download('stopwords')
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
s=set(stopwords.words('english'))

start = 'C:/Users/gony/Desktop/'

In [2]:
def remove_stopwords(text):
    return list(filter(lambda w: not w.lower() in s,text))

In [3]:

lemmatizer = WordNetLemmatizer()
def create_nodes_edges(data):
    # creating emtpy df
    output_graph = pd.DataFrame(columns=['source', 'target'])
    
    # iterating over each row 
    for i, row in tqdm(data.iterrows()):
        temp = row['words_sentences']
        
        # iterating over each sentence, since we are creating an each between two words
        # if the appeared together in the same sentence
        for sen in temp:
            pos_tags = pos_tag(sen)
            # creates list of permutation of 2 for all the words in the sentence
            lemmatized_words = []
            for word, pos in pos_tags:
                if pos.startswith('V'):  # Verbs
                    lemma = lemmatizer.lemmatize(word, pos='v')
                elif pos.startswith('N'):  # Nouns
                    lemma = lemmatizer.lemmatize(word, pos='n')
                else:
                    lemma = word  # Keep the word as is
                lemmatized_words.append(lemma)

            # Join the lemmatized words back into a sentence
            # creates list of permutation of 2 for all the words in the sentence
            temp_df = pd.DataFrame(data =list(combinations([x.lower() for x in lemmatized_words], 2)), columns=['source', 'target'])
            output_graph = output_graph.append(temp_df, ignore_index = True)
    
#     # groupby source and target and count all the apperance
#     output_graph['count']=output_graph.groupby('source')['target'].transform('count')#all rows ; 
    count_series = output_graph.groupby(['source', 'target']).size()

#     # remove duplicates
#     output_graph.drop_duplicates(inplace=True)
    new_df = count_series.to_frame(name = 'weight').reset_index()

    return new_df

In [6]:
path = start+'data/9_11_2001.zip'
with ZipFile(path, "r") as zip_ref:
    for f in zip_ref.namelist():
        idx = [i for i, ltr in enumerate(f) if ltr == '.']
        num_day = int(f[:idx[0]])
        num_month = int(f[idx[0]+1:idx[1]])
        if (num_month==9 and num_day>=6 and num_day<=26):
            print(f)
            day = pd.read_csv(zip_ref.open(f))

            day['temp'] = day.apply(lambda row: re.sub(r'\b[A-Z]+\b', '', row['text']), axis=1)
            # split each text by sentences, receiving list of strings each string is a sentence
            # in the original text.
            day['sentences'] = day.temp.str.split('.')

            # removing punctiuation marks from each sentence
            day['sentences'] = [[s.translate(str.maketrans('', '', string.punctuation)) for s in t] for t in day.sentences]

            # split each sentence to words, reveiving list of lists,
            # each sublist has all the words in  original sentence after removing stopwords
            day['words_sentences'] =[[list(filter(lambda w: not w.lower() in s,sen.split())) for sen in text] for text in day.sentences]

            # splitting all words in text to a list of words removing stopwords
            # (i forgot to remove punctuation marks but for now we aren't using this so i'll fix in later)
            day['words_text'] = [remove_stopwords(t.split()) for t in day.temp]
        #     print(Counter(day['words_text'][0]).most_common(5))
        #     print(day['sentences'][0][0])
        #     break
            create_nodes_edges(day).to_csv(start+'data/new/9_11_2001/nodes' + f)


10.09.2001.csv


102it [01:49,  1.07s/it]


11.09.2001.csv


110it [02:27,  1.34s/it]


12.09.2001.csv


...

102it [01:37,  1.05it/s]


8.09.2001.csv


54it [01:22,  1.54s/it]


9.09.2001.csv


54it [00:46,  1.15it/s]


In [10]:
path = start+'data/iraq_3_20_2003.zip'
with ZipFile(path, "r") as zip_ref:
    for f in zip_ref.namelist():
        idx = [i for i, ltr in enumerate(f) if ltr == '.']
        num_day = int(f[:idx[0]])
        num_month = int(f[idx[0]+1:idx[1]])
        if ((num_month==3 and num_day>=16) or (num_month==4 and num_day<=2)) and not os.path.exists(start+'data/new/iraq_3_20_2003/nodes'+f):
            print(f)
            day = pd.read_csv(zip_ref.open(f))
            day['sentences'] = day.text.str.split('.')
            day = day[day['sentences'].notna()]
            day['sentences'] = [[s.translate(str.maketrans('', '', string.punctuation)) for s in t] for t in day.sentences]
            day['words_sentences'] =[[list(filter(lambda w: not w.lower() in s,sen.split())) for sen in text] for text in day.sentences]
            day['words_text'] = [remove_stopwords(t.split()) for t in day.text]
            create_nodes_edges(day).to_csv(start+'data/new/iraq_3_20_2003/nodes' + f)

78it [03:17,  2.53s/it]


16.03.2003.csv


48it [01:44,  2.18s/it]


17.03.2003.csv


62it [02:10,  2.10s/it]


...

43it [04:50,  6.75s/it]


30.03.2003.csv


47it [03:47,  4.85s/it]


31.03.2003.csv


66it [01:33,  1.42s/it]


In [10]:
path = start + 'data/katrina_08_2005.zip'
with ZipFile(path, "r") as zip_ref:
    for f in zip_ref.namelist():
        idx = [i for i, ltr in enumerate(f) if ltr == '.']
        num_day = int(f[:idx[0]])
        num_month = int(f[idx[0]+1:idx[1]])
        if ((num_month==9 and num_day<=6) or (num_month==8 and num_day>=18)):
            print(f)
            day = pd.read_csv(zip_ref.open(f))
            day['sentences'] = day.text.str.split('.')
            day = day[day['sentences'].notna()]
            day['sentences'] = [[s.translate(str.maketrans('', '', string.punctuation)) for s in t] for t in day.sentences]
            day['words_sentences'] =[[list(filter(lambda w: not w.lower() in s,sen.split())) for sen in text] for text in day.sentences]
            day['words_text'] = [remove_stopwords(t.split()) for t in day.text]
            create_nodes_edges(day).to_csv(start+'data/new/katrina_08_2005/nodes' + f)


1.09.2005.csv


28it [02:23,  5.13s/it]


18.08.2005.csv


23it [01:46,  4.65s/it]


19.08.2005.csv


...

12it [00:46,  3.87s/it]


5.09.2005.csv


27it [01:42,  3.80s/it]


6.09.2005.csv


29it [01:52,  3.88s/it]


In [11]:
path = start+'data/binladin_02_05_2011/'
for f in os.listdir(path):
    idx = [i for i, ltr in enumerate(f) if ltr == '.']
    num_day = int(f[:idx[0]])
    num_month = int(f[idx[0]+1:idx[1]])
    if ((num_month==5 and num_day<=17) or (num_month==4 and num_day>=27)):
        print(f)
        day = pd.read_csv(path+f)
        day['sentences'] = day.text.str.split('.')
        day = day[day['sentences'].notna()]
        day['sentences'] = [[s.translate(str.maketrans('', '', string.punctuation)) for s in t] for t in day.sentences]
        day['words_sentences'] =[[list(filter(lambda w: not w.lower() in s,sen.split())) for sen in text] for text in day.sentences]
        day['words_text'] = [remove_stopwords(t.split()) for t in day.text]
        create_nodes_edges(day).to_csv(start+'data/new/binladin_02_05_2011/nodes' + f)

1.05.2011.csv


17it [01:44,  6.13s/it]


10.05.2011.csv


26it [05:05, 11.76s/it]


11.05.2011.csv


...

20it [02:15,  6.77s/it]


8.05.2011.csv


14it [01:16,  5.43s/it]


9.05.2011.csv


26it [03:16,  7.55s/it]


In [12]:
path = start+'data/boston_15_04_2013/'
for f in os.listdir(path):
    idx = [i for i, ltr in enumerate(f) if ltr == '.']
    num_day = int(f[:idx[0]])
    num_month = int(f[idx[0]+1:idx[1]])
    if (num_month==4 and (num_day>= 11 or num_day<=30)):
        print(f)
        day = pd.read_csv(path+f)
        day['sentences'] = day.text.str.split('.')
        day = day[day['sentences'].notna()]
        day['sentences'] = [[s.translate(str.maketrans('', '', string.punctuation)) for s in t] for t in day.sentences]
        day['words_sentences'] =[[list(filter(lambda w: not w.lower() in s,sen.split())) for sen in text] for text in day.sentences]
        day['words_text'] = [remove_stopwords(t.split()) for t in day.text]
        create_nodes_edges(day).to_csv(start+'data/new/boston_15_04_2013/nodes' + f)


11.04.2013.csv


39it [03:18,  5.10s/it]


12.04.2013.csv


40it [03:37,  5.43s/it]


13.04.2013.csv


...

14it [01:13,  5.22s/it]


29.04.2013.csv


39it [03:30,  5.40s/it]


30.04.2013.csv


39it [02:58,  4.58s/it]


In [19]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
stemmer.stem("John")

'john'

In [6]:
data = pd.read_csv(start+'data/boston_15_04_2013/11.04.2013.csv')
data['sentences'] = data.text.str.split('.')
data = data[data['sentences'].notna()]
data['sentences'] = [[s.translate(str.maketrans('', '', string.punctuation)) for s in t] for t in data.sentences]
data['words_sentences'] =[[list(filter(lambda w: not w.lower() in s,sen.split())) for sen in text] for text in data.sentences]
data['words_text'] = [remove_stopwords(t.split()) for t in data.text]
for i, row in tqdm(data.iterrows()):
    temp = row['words_sentences']
    break

0it [00:00, ?it/s]


['ANDERSON', 'COOPER', 'CNN', 'ANCHOR', 'Good', 'evening', 'everyone']
[('ANDERSON', 'NNP'), ('COOPER', 'NNP'), ('CNN', 'NNP'), ('ANCHOR', 'NNP'), ('Good', 'NNP'), ('evening', 'NN'), ('everyone', 'NN')]
['ANDERSON', 'COOPER', 'CNN', 'ANCHOR', 'Good', 'evening', 'everyone']
